In [1]:
using Flux
using Flux: Data.DataLoader
using Flux: onehotbatch,onecold,crossentropy
using Flux: @epochs
using Statistics
using MLDatasets

In [2]:
x_train_temp,y_train_temp = MNIST.traindata();
x_test_temp,y_test_temp = MNIST.testdata();

In [3]:
println("The size of the training images are $(size(x_train_temp)) and the size of training labels is $(size(y_train_temp))")
print("The size of the testing images are $(size(x_test_temp)) and the size of testing labels is $(size(y_test_temp))")

The size of the training images are (28, 28, 60000) and the size of training labels is (60000,)
The size of the testing images are (28, 28, 10000) and the size of testing labels is (10000,)

In [5]:
x_train = Flux.unsqueeze(x_train_temp[:,:,1:1000],3)
x_test = Flux.unsqueeze(x_test_temp,3)

y_train = onehotbatch(y_train_temp[1:1000],0:9)
y_test = onehotbatch(y_test_temp,0:9)

#Displaying the updated shape of the elements

println("The size of the training images are $(size(x_train)) and the size of training labels is $(size(y_train))")
print("The size of the testing images are $(size(x_test)) and the size of testing labels is $(size(y_test))")

train_data = DataLoader(x_train, y_train, batchsize=128);

The size of the training images are (28, 28, 1, 1000) and the size of training labels is (10, 1000)
The size of the testing images are (28, 28, 1, 10000) and the size of testing labels is (10, 10000)

In [6]:
layer1 = Conv((3,3),1=>8,relu,stride=2)
layer2 = Conv((3,3),8=>16,relu)
layer3 = Conv((3,3),16=>32,relu)
layer4 = GlobalMaxPool()
layer5 = flatten
layer6 = Dense(32,10)
layer7 = softmax;

In [7]:
layers = [layer1, layer2, layer3, layer4, layer5, layer6, layer7]
x = x_train

for l in layers
    println("The layer is $l")
    println("The size of the input layer is $(size(x))")
    x = l(x)
    println("The size of the output layer is $(size(x))")
    println("-"^30)
end

The layer is Conv((3, 3), 1=>8, relu)
The size of the input layer is (28, 28, 1, 1000)


┌ Warning: Slow fallback implementation invoked for conv!  You probably don't want this; check your datatypes.
│   yT = Float32
│   T1 = FixedPointNumbers.Normed{UInt8,8}
│   T2 = Float32
└ @ NNlib C:\Users\Saad.LAKES\.julia\packages\NNlib\fxLrD\src\conv.jl:206


The size of the output layer is (13, 13, 8, 1000)
------------------------------
The layer is Conv((3, 3), 8=>16, relu)
The size of the input layer is (13, 13, 8, 1000)
The size of the output layer is (11, 11, 16, 1000)
------------------------------
The layer is Conv((3, 3), 16=>32, relu)
The size of the input layer is (11, 11, 16, 1000)
The size of the output layer is (9, 9, 32, 1000)
------------------------------
The layer is GlobalMaxPool()
The size of the input layer is (9, 9, 32, 1000)
The size of the output layer is (1, 1, 32, 1000)
------------------------------
The layer is flatten
The size of the input layer is (1, 1, 32, 1000)
The size of the output layer is (32, 1000)
------------------------------
The layer is Dense(32, 10)
The size of the input layer is (32, 1000)
The size of the output layer is (10, 1000)
------------------------------
The layer is softmax
The size of the input layer is (10, 1000)
The size of the output layer is (10, 1000)
------------------------------

In [8]:
# model = Chain(layer1, layer2, layer3, layer4, layer5,layer6, layer7)
# # model = Chain(Dense(784,32),Dense(32,10),softmax)
# L(x,y) = Flux.crossentropy(model(x),y)
# opt = Flux.Optimise.ADAM()
# ps = Flux.params(model)

# evalcb() = @show(L(x_train, y_train))
# throttled_cb = Flux.throttle(evalcb, 1)

# # callback() = push!(loss_vector, L(x_train, y_train))
# # loss_vector = Vector{Float32}()
# print("Training started")

# @epochs 100 Flux.train!(L,ps,train_data,opt, cb = throttled_cb)

In [10]:
model1 = Chain(layer1, layer2, layer3, layer4, layer5,layer6, layer7)
L1(x,y) = Flux.crossentropy(model1(x),y)
opt1 = Flux.Optimise.ADAM()
ps1 = Flux.params(model1)

total_step = 100

for step in 1:total_step
    Flux.train!(L1,ps1,train_data,opt1)
    println("EPOCH NUMBER: $step")
    println("Training Loss is $(L1(x_train,y_train))")
    println("Validation Loss is $(L1(x_test,y_test))")
    println("-"^50)
end

EPOCH NUMBER: 1
Training Loss is 0.05334028
Validation Loss is 0.41607904
--------------------------------------------------
EPOCH NUMBER: 2
Training Loss is 0.033022523
Validation Loss is 0.43631163
--------------------------------------------------
EPOCH NUMBER: 3
Training Loss is 0.019734679
Validation Loss is 0.39910105
--------------------------------------------------
EPOCH NUMBER: 4
Training Loss is 0.020999355
Validation Loss is 0.40572324
--------------------------------------------------
EPOCH NUMBER: 5
Training Loss is 0.017284023
Validation Loss is 0.41206014
--------------------------------------------------
EPOCH NUMBER: 6
Training Loss is 0.01517956
Validation Loss is 0.4153704
--------------------------------------------------
EPOCH NUMBER: 7
Training Loss is 0.015405611
Validation Loss is 0.40452024
--------------------------------------------------
EPOCH NUMBER: 8
Training Loss is 0.013933814
Validation Loss is 0.42643934
----------------------------------------------

EPOCH NUMBER: 66
Training Loss is 0.0017542881
Validation Loss is 0.46449238
--------------------------------------------------
EPOCH NUMBER: 67
Training Loss is 0.0016138303
Validation Loss is 0.46519887
--------------------------------------------------
EPOCH NUMBER: 68
Training Loss is 0.0015471834
Validation Loss is 0.46548364
--------------------------------------------------
EPOCH NUMBER: 69
Training Loss is 0.001499524
Validation Loss is 0.46451998
--------------------------------------------------
EPOCH NUMBER: 70
Training Loss is 0.0014587882
Validation Loss is 0.46361822
--------------------------------------------------
EPOCH NUMBER: 71
Training Loss is 0.0014248887
Validation Loss is 0.46363208
--------------------------------------------------
EPOCH NUMBER: 72
Training Loss is 0.0013936943
Validation Loss is 0.46416163
--------------------------------------------------
EPOCH NUMBER: 73
Training Loss is 0.0013647692
Validation Loss is 0.46468183
----------------------------

In [38]:
accuracy(model(x_test),y_test)

0.71

In [114]:
foo=reshape(x_train_temp,(784,60000));

In [117]:
size(x_train_temp)

(28, 28, 60000)

In [120]:
preprocess(p) = vec(Float64.(p))
foo = preprocess.(x_train_temp[1:100])

LoadError: MethodError: no method matching vec(::Float64)
Closest candidates are:
  vec(!Matched::LinearAlgebra.Transpose{T,var"#s828"} where var"#s828"<:(AbstractArray{T,1} where T) where T) at C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.5\LinearAlgebra\src\adjtrans.jl:226
  vec(!Matched::LinearAlgebra.Adjoint{var"#s828",var"#s8281"} where var"#s8281"<:(AbstractArray{T,1} where T) where var"#s828"<:Real) at C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.5\LinearAlgebra\src\adjtrans.jl:227
  vec(!Matched::SparseArrays.AbstractSparseArray{Tv,Ti,1} where Ti where Tv) at C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.5\SparseArrays\src\sparsevector.jl:911
  ...

In [126]:
length.(x_train_temp)

28×28×60000 Array{Int64,3}:
[:, :, 1] =
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1 